In [2]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import time
from random import randrange
import streamlit as st

import utils.func as func

In [4]:
hutid = [3]
date =  '19.07.2024'


headers = {
    'Accept': 'application/json, text/javascript, */*; q=0.01',
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36'
}

s = requests.Session()
s.headers.update(headers)
#loop over all huts, create a dataframe for each hut
#initialize dataframes
df_list = []

for hut in hutid:
    r = s.get(f'https://www.alpsonline.org/reservation/calendar?hut_id={hut}')
    r = s.get(f'https://www.alpsonline.org/reservation/selectDate?date={date}')
    json_list = []
    for i, value in r.json().items():
        json_list.append(value[0])
    df = pd.DataFrame(json_list)
    df['hutid'] = hut
    df_list.append(df)


#reduce dataframes to relevant columns, add hutid and calculate occupation
df_red_list = []
for df in df_list:
    df_red = df[['hutid', 'reservationDate', 'freeRoom']].copy()
    df_red['occupation'] = (100 - df_red['freeRoom'].copy() / df['totalRoom'].copy() * 100)
    df_red['Service'] = df['bedCategoryType'].copy()
    df_red_list.append(df_red)

#concatenate dataframes
df_con = pd.concat(df_red_list)
print(df_con)


    hutid reservationDate  freeRoom  occupation Service
0       3      19.07.2024        15   82.142857    ROOM
1       3      20.07.2024        20   76.190476    ROOM
2       3      21.07.2024        59   29.761905    ROOM
3       3      22.07.2024        44   47.619048    ROOM
4       3      23.07.2024        42   50.000000    ROOM
5       3      24.07.2024        34   59.523810    ROOM
6       3      25.07.2024        31   63.095238    ROOM
7       3      26.07.2024        22   73.809524    ROOM
8       3      27.07.2024        10   88.095238    ROOM
9       3      28.07.2024        52   38.095238    ROOM
10      3      29.07.2024        38   54.761905    ROOM
11      3      30.07.2024        53   36.904762    ROOM
12      3      31.07.2024        46   45.238095    ROOM
13      3      01.08.2024         0  100.000000    ROOM


In [5]:
df

,hutBedCategoryId,bedCategoryId,bedCategoryType,bookingEnabled,closed,events,freeRoom,futureHutOccupancyShown,reservationDate,totalRoom,contingentId,contingentText,contingentTitle,unserviced,unservicedWaitingList,hutDefaultLanguage,reservedRoomsRatio,eventsAsString,hutid
0,64,1,ROOM,True,False,[],15,False,19.07.2024,84,None,None,None,False,False,de_CH,0.821429,,3
1,64,1,ROOM,True,False,[],20,False,20.07.2024,84,None,None,None,False,False,de_CH,0.761905,,3
2,64,1,ROOM,True,False,[],59,False,21.07.2024,84,None,None,None,False,False,de_CH,0.297619,,3
3,64,1,ROOM,True,False,[],44,False,22.07.2024,84,None,None,None,False,False,de_CH,0.476190,,3
4,64,1,ROOM,True,False,[],42,False,23.07.2024,84,None,None,None,False,False,de_CH,0.500000,,3
5,64,1,ROOM,True,False,[],34,False,24.07.2024,84,None,None,None,False,False,de_CH,0.595238,,3
6,64,1,ROOM,True,False,[],31,False,25.07.2024,84,None,None,None,False,False,de_CH,0.630952,,3
7,64,1,ROOM,True,False,[],22,False,26.07.2024,84,None,None,None,False,False,de_CH,0.738095,,3
8,64,1,ROOM,True,False,[],10,False,27.07.2024,84,None,None,None,False,False,de_CH,0.880952,,3
9,64,1,ROOM,True,False,[],52,False,28.07.2024,84,None,None,None,False,False,de_CH,0.380952,,3


In [6]:
#Read csv file with hut information
#contains: hutid, hutname, altitude
df_huts = pd.read_excel('hutInfo.xlsx')
print(df_huts)

     Hut ID                                      Hut Name Altitude  \
0         1  Blüemlisalphütte SAC, SAC Sektion Blümlisalp     2840   
1         2             Glecksteinhütte SAC, SAC Burgdorf     2317   
2         3      Capanna da l'Albigna CAS, SAC Hoher Rohn     2336   
3         4                    Kesch-Hütte SAC, SAC Davos     2625   
4         5                 Lidernenhütte SAC, SAC Mythen     1727   
..      ...                                           ...      ...   
315     663                               Berghaus Bäregg     1775   
316     664                        Berggasthaus Dürrboden     2007   
317     665                Guggihütte SAC, SAC Interlaken     2791   
318     668                           Cabane de la Barmaz     2458   
319     670                                 Alpe Rompiago     1290   

                                Coordinates  \
0                         625.565 / 151.040   
1                         650.400 / 163.979   
2                 

In [7]:
a = df_huts['Coordinates'].apply(lambda x: x.split(','))
a

0                           [625.565 / 151.040]
1                           [650.400 / 163.979]
2                           [770.675 / 133.375]
3                           [786.561 / 168.870]
4                           [695.500 / 199.000]
                         ...                   
315    [46.600492931355355 / 8.060145726418344]
316                [46°43'15.2"N / 9°55'18.6"E]
317                         [641 080 / 156 970]
318                  [2'595'925/ 1'101'119, 88]
319                         [720'488 / 105'668]
Name: Coordinates, Length: 320, dtype: object

In [27]:
def FindBeds(date, NumPeople = 1, nights = 1, SACOnly = False):
    #Input:
        #first night of the reservation
        #look for all huts, not just SAC huts
        #number of people in the reservation
        #number of nights in the reservation
    #Output:
        #dataframe with all suitable huts, their occupation and the number of beds available

    #load hut information (hutid, hutname, altitude)
    if SACOnly:
        df_huts = pd.read_excel('HutInfo.xlsx')
    else:
        df_huts = pd.read_excel('HutInfoSAC.xlsx')

    #initialize pandas dataframe
    df_freeHuts = pd.DataFrame(columns = ['Hut ID', 'Hut Name', 'Altitude', 'Free Beds', 'Capacity', 'Service', 'Reservation Link'])

    #initialize session to grab data from alpsonline
    headers = {
        'Accept': 'application/json, text/javascript, */*; q=0.01',
       'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36'
    }

    s = requests.Session()
    s.headers.update(headers)

    hutCounter = 0 #count how many huts we've found
    
    #loop through all hutIDs
    for index, hut in enumerate(df_huts['Hut ID']):
        r = s.get(f'https://www.alpsonline.org/reservation/calendar?hut_id={hut}')
        r = s.get(f'https://www.alpsonline.org/reservation/selectDate?date={date}')
        json_list = []
        for i, value in r.json().items():
            json_list.append(value[0])
        df = pd.DataFrame(json_list)
        df['hutid'] = hut
        df_list.append(df)
        #go through df, check if there are enough beds available
        for j in range(0,nights):
            hasfreerooms = True
            if df['freeRoom'][j] < NumPeople:
                hasfreerooms = False
                break
        if hasfreerooms:
            #add hutid, hutname, altiude, occupation, freeRoom, Service to df_freeHuts
            isunattended = df['bedCategoryType'][0]
            if isunattended == 'Unattended':
                isunattended = 'Self-Service'
            else:
                isunattended = 'Full-Service'
            df_freeHuts.loc[hutCounter] = [df_huts['Hut ID'][index], df_huts['Hut Name'][index], df_huts['Altitude'][index], df['freeRoom'][0], df['reservedRoomsRatio'][0]*100, df['contingentTitle'][0], df_huts['Reservation Link'][index]]
            hutCounter += 1
    
    return df_freeHuts

In [30]:
test = FindBeds(date = '11.07.2024', NumPeople=2, nights=1, SACOnly = False)
print(test)

   Hut ID                                      Hut Name  Altitude  Free Beds  \
0       1  Blüemlisalphütte SAC, SAC Sektion Blümlisalp      2840         11   
1       2             Glecksteinhütte SAC, SAC Burgdorf      2317         46   
2       3      Capanna da l'Albigna CAS, SAC Hoher Rohn      2336         49   
3       4                    Kesch-Hütte SAC, SAC Davos      2625         22   

    Capacity Service                                   Reservation Link  
0  87.777778    None  https://www.alpsonline.org/reservation/calenda...  
1  28.125000    None  https://www.alpsonline.org/reservation/calenda...  
2  41.666667    None  https://www.alpsonline.org/reservation/calenda...  
3  72.500000    None  https://www.alpsonline.org/reservation/calenda...  


In [37]:
st.dataframe(test)

DeltaGenerator()

In [10]:
#look up reservations

hutid = 680
date = '10.07.2024'

r = s.get(f'https://www.alpsonline.org/reservation/calendar?hut_id={hut}')
r = s.get(f'https://www.alpsonline.org/reservation/selectDate?date={date}')
json_list = []
for i, value in r.json().items():
    json_list.append(value[0])
df = pd.DataFrame(json_list)
df['hutid'] = hut
df_list.append(df)

print(df)

    hutBedCategoryId  bedCategoryId bedCategoryType  bookingEnabled  closed  \
0               1350             43      UNSERVICED            True   False   
1               1350             43      UNSERVICED            True   False   
2               1350             43      UNSERVICED            True   False   
3               1350             43      UNSERVICED            True   False   
4               1350             43      UNSERVICED            True   False   
5               1350             43      UNSERVICED            True   False   
6               1350             43      UNSERVICED            True   False   
7               1350             43      UNSERVICED            True   False   
8               1350             43      UNSERVICED            True   False   
9               1350             43      UNSERVICED            True   False   
10              1350             43      UNSERVICED            True   False   
11              1350             43      UNSERVICED 

In [16]:
df_freeHuts = pd.DataFrame(columns = ['hutid', 'hutname', 'altitude', 'occupation', 'freeRoom', 'Service'])
df_freeHuts = df_freeHuts.append({'hutid': df_huts['Hut ID'][hut], 'hutname': df_huts['Hut Name'][hut], 'altitude': df_huts['Altitude'][hut], 'occupation': df['occupation'][0], 'freeRoom': df['freeRoom'][0], 'Service': df['bedCategoryType'][0]}, ignore_index = True)


AttributeError: 'DataFrame' object has no attribute 'append'

In [6]:
def findHutIDsDebug(hutidLimit = 1000):
    #goes through all hut ids and checks if the hut exists
    #if the hut exists, it saves the hut id, altitude and coordinates in a dataframe
    #if the hut does not exist, it is skipped
    #returns a dataframe with hut information
    hutInfo = pd.DataFrame(columns = ['Hut ID', 'Hut Name', 'Altitude', 'Coordinates', 'Reservation Link'])
    hutInfoSAC = pd.DataFrame(columns = ['Hut ID', 'Hut Name', 'Altitude', 'Coordinates', 'Reservation Link'])
    hutInfoCH = pd.DataFrame(columns = ['Hut ID', 'Hut Name', 'Altitude', 'Coordinates', 'Reservation Link'])
    #parameters
    date = '11.07.2027' #pick a date well into the future
    hutCounter = 0
    hutCounterSAC = 0
    hutCounterCH = 0 
    #------------change hutidLmit to 1000 to check all huts-----------

    
    #initialize session
    headers = {
        'Accept': 'application/json, text/javascript, */*; q=0.01',
        'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36'
        }
    s = requests.Session()
    s.headers.update(headers)


    for hutid in range(39,hutidLimit):
        #wait a bit to not overload the server
        time.sleep(0.5*randrange(0,10))
        page = s.get(f'https://www.alpsonline.org/reservation/calendar?hut_id={hutid}&selectDate={date}&lang=en')
        soup = bs(page.content, 'html.parser')
        
        name_check = ['DAV', 'AIV', 'CAI', 'Alpenverein', 'ZZZ', 'AVS']

        #If the hut does not exist or is not activated, the error message is different
        erm = soup.find_all('div', {'class': 'errorsMessage'})
        noHutError = f'<div class="errorsMessage">The requested hut [{hutid}] cannot be found. Please check your parameters.</div>'
        hutNotActivatedError = f'<div class="errorsMessage">This hut profile is not activated. Bookings cannot be entered at the moment. Please contact the administrator.</div>'
        if str(erm[3]) == noHutError:
            #if the hut does not exist
            print(f'Hut with id {hutid} not found')
            continue
        elif str(erm[3]) == hutNotActivatedError:
            #if the hut is not activated
            print(f'Hut with id {hutid} not activated')
            continue
        else:
            print(f'Hut with id {hutid} found')
            hutInfBS = soup.body.find('div', attrs={'class': 'info'}).find_all('span')
            hutname = soup.body.find('div', {'class': 'info'}).find_all('h4')[0].text

        if hutInfBS[3].text[-1] == 'm':
            altitude = hutInfBS[3].text[24:-2]
        else:
            altitude = hutInfBS[3].text[24:]

        #check if the hut is in Switzerland        
        swissHut = True
        for name in name_check:
            if name in hutname:
                swissHut = False
                break

        if swissHut:
            hutInfoCH.loc[hutCounter] = [hutid, hutname, altitude, hutInfBS[4].text[13:], f'https://www.alpsonline.org/reservation/calendar?hut_id={hutid}&selectDate={date}&lang=en']
            hutCounterCH += 1

        #one dataframe with SAC huts only
        if 'SAC' in hutname and swissHut:
            hutInfoSAC.loc[hutCounter] = [hutid, hutname, altitude, hutInfBS[4].text[13:], f'https://www.alpsonline.org/reservation/calendar?hut_id={hutid}&selectDate={date}&lang=en']
            hutCounterSAC += 1
        #one dataframe with all huts
        hutInfo.loc[hutCounter] = [hutid, hutname, altitude, hutInfBS[4].text[13:], f'https://www.alpsonline.org/reservation/calendar?hut_id={hutid}&selectDate={date}&lang=en']
        hutCounter += 1


    return hutInfo, hutInfoSAC, hutInfoCH


In [7]:
[a,b,c] = findHutIDsDebug(41)

Hut with id 39 found
Hut with id 40 not activated


In [8]:
print(a)
print(b)
print(c)

   Hut ID                       Hut Name Altitude        Coordinates  \
0      39  Glattalphütte SAC, SAC Mythen     1896  709.410 / 197.200   

                                    Reservation Link  
0  https://www.alpsonline.org/reservation/calenda...  
   Hut ID                       Hut Name Altitude        Coordinates  \
0      39  Glattalphütte SAC, SAC Mythen     1896  709.410 / 197.200   

                                    Reservation Link  
0  https://www.alpsonline.org/reservation/calenda...  
   Hut ID                       Hut Name Altitude        Coordinates  \
0      39  Glattalphütte SAC, SAC Mythen     1896  709.410 / 197.200   

                                    Reservation Link  
0  https://www.alpsonline.org/reservation/calenda...  
